# 패키지 설치

In [ ]:
!pip install ratsnlp

In [ ]:
!pip install --upgrade pytorch-lightning

# 환경 설정

In [4]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments

모델 hyperparameter와 저장 위치 등 설정 정보를 선언

In [5]:
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/content/drive/Othercomputers/내 컴퓨터/Chapter 8. Sentence Generation/checkpoint-generation",
)

downstream_model_checkpoint_fpath: /content/drive/Othercomputers/내 컴퓨터/Chapter 8. Sentence Generation/checkpoint-generation/epoch=1-val_loss=2.29.ckpt


# 모델 불러오기

파인튜닝을 마친 모델 불러오기

In [6]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel

In [7]:
# 체크포인트 로드
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

# GPT2 설정 로드
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)

model = GPT2LMHeadModel(pretrained_model_config)        # 모델 초기화
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})     # 체크포인트 주입
model.eval()        # 평가모드

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


토크나이저 준비

In [8]:
from transformers import PreTrainedTokenizerFast

In [9]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    bos_token='</s>',       # 문장 시작토큰
    eos_token='</s>',       # 문장 마지막토큰
    unk_token='<unk>',      # 어휘에 없는 토큰
    pad_token='<pad>',      # 크기 맞추기 토큰
    mask_token='<mask>'     # 마스킹 토큰
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


# 인퍼런스 함수

In [10]:
def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        # 입력 문장 토큰화 및 인덱싱 후 텐서로 변환
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        # 평가 모드로 Gradient 업데이트 안함
        with torch.no_grad():
            # input_ids 뒤에 이어지는 토큰 생성
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])   # gererated_ids를 텍스트로 변환

    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

# 웹서비스 개시

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구이다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 입력해야 한다.

In [11]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

인퍼런스 함수를 Flask를 통해 웹서비스로 만든다.

In [12]:
from ratsnlp.nlpbook.generation import get_web_service_app

In [13]:
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.generation.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9d26-35-233-172-243.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Apr/2022 05:52:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 05:52:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Apr/2022 05:52:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 05:53:54] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 05:54:29] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 05:54:40] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 05:54:52] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 05:54:57] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2022 05:55:19] "POST /api HTTP/1.1" 200 -
